# Plot Delineaion 
 At first will put inplace all the requirede documents necessary foro the extractionof plot from satellie imagery 

 

In [17]:
#!git clone https://github.com/njr3/Mask_RCNN.git
#%tensorflow_version 1.x
#!pip install --upgrade h5py==2.10.0
#!git clone https://github.com/pysource7/Mask_RCNN
import sys
#sys.path.append("/content/Mask_RCNN")
#from train_mask_rcnn_demo import *
%matplotlib inline

In [18]:
cd Mask_RCNN

[WinError 2] The system cannot find the file specified: 'Mask_RCNN'
C:\Users\Steven\Downloads\new_automation-20220128T083924Z-001\new_automation\Mask_RCNN


In [19]:
!nvidia-smi

Mon Jan 31 16:24:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.09       Driver Version: 511.09       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 3000    WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   54C    P8     6W /  N/A |    583MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
with open('mrcnn/model.py') as f:
    model_file = f.read()

with open('mrcnn/model.py', 'w') as f:
    model_file = model_file.replace("self.keras_model = self.build(mode=mode, config=config)",
                                    "self.keras_model = self.build(mode=mode, config=config)\n        self.keras_model.metrics_tensors = []")
    f.write(model_file)

In [21]:
#from osgeo import gdal
import scipy
import skimage
import numpy as np
import os
import subprocess
import numpy as np
from shapely.geometry import shape
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd

from affine import Affine
from skimage.morphology import square, erosion, dilation
from tqdm.auto import tqdm
import math

from itertools import product

import sys
import skimage
import os
from os import listdir
from os.path import join
from os.path import isfile
import geopandas
from shapely import wkt
from shapely.geometry import Polygon
import pandas as pd


In [22]:
#import skimage.io as io
import matplotlib.image as mpimg
import os
from os import listdir
from os.path import join
from os.path import isfile
import sys

In [23]:
import theano

In [24]:

import rasterio
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
#from osgeo import gdal, ogr, osr
#import rasterio
#from rasterio.plot import show
#import rasterio as rio
#from rasterio import windows
#from rasterio import features

ImportError: DLL load failed: The specified module could not be found.

In [25]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from tkinter import *
from tkinter import Tcl


In [ ]:


def interior_clean(path,out_path,eps,min_area=None):
    shape=geopandas.read_file(path)
    list_interiors = []
    out=[]
    r=[]
    #eps = 10
    for polygon in shape['geometry']:
        for interior in polygon.interiors:
            p = Polygon(interior)    
            if p.area > eps:
                list_interiors.append(interior)
        new_polygon = Polygon(polygon.exterior.coords, holes=list_interiors)
        out.append(new_polygon)
    if min_area is not None:
        for x in out:
            if x.area>min_area:
                r.append(x)
    df3 = pd.DataFrame([ a for a in r],columns=['geometry'])
    gdf = geopandas.GeoDataFrame(df3)
    gdf.to_file(filename=out_path, driver='GeoJSON')


In [ ]:
import os
from os import listdir
from os.path import join
from os.path import isfile
from tkinter import Tcl
from osgeo import gdal

def conversion(input_path ,path):
  input= [f for f in listdir(input_path) if isfile(join(input_path, f)) and  f.endswith(".tif")]
  inp=list(Tcl().call('lsort', '-dict',input))
  for x in inp:
    original_path=input_path+x
    output_path= path+ x[:-4]+'.jpg'
    #!gdal_translate -of JPEG -ot Byte  original_path output_path
    os.system('gdal_translate -of JPEG -ot Byte {cd} {v} '.format(cd=original_path,v=output_path))

In [ ]:
def filtering(path,save_path):
    "This function is to filter the image directly after it is out from the  deep learning  "
    nlyTIFF = [os.path.join( path, f) for f in listdir(path) if isfile(join(path, f)) and  f.endswith(".jpg")]
    nlyTIFF=Tcl().call('lsort', '-dict',nlyTIFF)
    if len(nlyTIFF)>=2:
        for i in range(len(nlyTIFF)):
            grayscale=io.imread(nlyTIFF[i],plugin='matplotlib')
            median_filtered = scipy.ndimage.median_filter(grayscale, size=3)
            threshold = skimage.filters.threshold_li(median_filtered)
            predicted = np.uint8(median_filtered > threshold) * 255
            io.imsave(os.path.join(save_path,"predict_{}.tif".format(int(i))), predicted )
    elif len(nlyTIFF)==1:
        grayscale=io.imread(nlyTIFF[0],plugin='matplotlib')
        median_filtered = scipy.ndimage.median_filter(grayscale, size=3)
        threshold = skimage.filters.threshold_li(grayscale)
        predicted = np.uint8(median_filtered > threshold) * 255
        io.imsave(os.path.join(save_path,"predict.tif"), predicted )
    else:
        print('No jpg file in the path given')


In [ ]:
import os
from os import listdir
from os.path import join
from os.path import isfile
import rasterio
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
import sys

#inputArg = sys.argv
#image_path = "output/prediction/"+inputArg[2] +'/'

#out_path = "output/merge/"+inputArg[2] +'/'



class Merge:
    def merge(image_path,  out_path):
        """Merge a series of raster .

    Arguments
    ---------
    image_path : Str
        Path to the series of raster.
    save_path : Str
        Path of to the merged raster.

    Returns
    -------
    A merge raster with it's metadata.
    
    Notes
    -----
    This functions depends on "rasterio" and it's submodules.
    """
        nlyTIFF = [os.path.join(image_path, f) for f in listdir(image_path) if isfile(join(image_path, f)) and  f.endswith(".tif")]
    # List for the source files
        src_files_to_mosaic = []
        for fp in nlyTIFF:
            src = rio.open(fp)
            src_files_to_mosaic.append(src) 
        mosaic, out_trans = merge(src_files_to_mosaic)
        # Copy the metadata
        out_meta = src.meta.copy()
        # Update the metadata
        out_meta.update({"driver": "GTiff",
                "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 })
        with rasterio.open(out_path, "w", **out_meta) as dest:
            dest.write(mosaic)



In [ ]:
from osgeo import gdal
import skimage
import numpy as np
from shapely.geometry import shape
from shapely.geometry import Polygon
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio import features






def Binary_mask_to_poly_geojson(imagepath,output_path, channel_scaling=None, reference_im=None,cr=None,
                          output_type='geojson', min_area=27,
                         bg_threshold=0, simplify=True,
                         tolerance=15, **kwargs):
    """Get polygons from an image mask.

    Arguments
    ---------
    pred_arr : :class:`numpy.ndarray`
        A 2D array of integers. Multi-channel masks are not supported, and must
        be simplified before passing to this function. Can also pass an image
        file path here.
    channel_scaling : :class:`list`-like, optional
        If `pred_arr` is a 3D array, this argument defines how each channel
        will be combined to generate a binary output. channel_scaling should
        be a `list`-like of length equal to the number of channels in
        `pred_arr`. The following operation will be performed to convert the
        multi-channel prediction to a 2D output ::

            sum(pred_arr[channel]*channel_scaling[channel])

        If not provided, no scaling will be performend and channels will be
        summed.
    reference_im : str, optional
        The path to a reference geotiff to use for georeferencing the polygons
        in the mask. Required if saving to a GeoJSON (see the ``output_type``
        argument), otherwise only required if ``do_transform=True``.
    output_path : str, optional
        Path to save the output file to. If not provided, no file is saved.
    output_type : ``'csv'`` or ``'geojson'``, optional
        If ``output_path`` is provided, this argument defines what type of file
        will be generated - a CSV (``output_type='csv'``) or a geojson
        (``output_type='geojson'``).
    min_area : int, optional
        The minimum area of a polygon to retain. Filtering is done AFTER
        any coordinate transformation, and therefore will be in destination
        units.
    bg_threshold : int, optional
        The cutoff in ``mask_arr`` that denotes background (non-object).
        Defaults to ``0``.
    simplify : bool, optional
        If ``True``, will use the Douglas-Peucker algorithm to simplify edges,
        saving memory and processing time later. Defaults to ``False``.
    tolerance : float, optional
        The tolerance value to use for simplification with the Douglas-Peucker
        algorithm. Defaults to ``0.5``. Only has an effect if
        ``simplify=True``.

    Returns
    -------
    gdf : :class:`geopandas.GeoDataFrame`
        A GeoDataFrame of polygons.

    """
    mask_arr = skimage.io.imread(fname=imagepath)
    if reference_im is None:
        with rasterio.open(imagepath) as ref:
            transform = ref.transform
            crs = ref.crs
            ref.close()
    else:
        with rasterio.open(reference_im) as ref:
            transform = ref.transform
            crs = ref.crs
            ref.close()
    mask = mask_arr > bg_threshold
    mask = mask.astype('uint8')


    polygon_generator = features.shapes(mask_arr,transform=transform,mask=mask)
    polygons = []
    values = []  # pixel values for the polygon in mask_arr
    for polygon, value in polygon_generator:
        p = shape(polygon).buffer(0.0)
        if p.area >= min_area:
            polygons.append(shape(polygon).buffer(0.0))
            values.append(value)

    polygon_gdf = gpd.GeoDataFrame({'geometry': polygons, 'value': values},
                                   crs=crs.to_wkt())
    if simplify:
        polygon_gdf['geometry'] = polygon_gdf['geometry'].apply(
            lambda x: x.simplify(tolerance=tolerance)
        )
    #changing the crs in case
    if cr is not None:
        polygon_gdf=polygon_gdf.to_crs(epsg=cr)
    # save output files
    if output_path is not None:
        if output_type.lower() == 'geojson':
            #if len(polygon_gdf) > 0:
            polygon_gdf.to_file(output_path, driver='GeoJSON')
            #else:
                #save_empty_geojson(output_path, polygon_gdf.crs.to_epsg())
        elif output_type.lower() == 'csv':
            polygon_gdf.to_csv(output_path, index=False)
        else:
            polygon_gdf.to_file(output_path)
    return polygon_gdf

In [ ]:
import math
#rom osgeo import gdal, ogr, osr
import rasterio
from rasterio.plot import show
import os
from itertools import product
import rasterio as rio
from rasterio import windows
import sys

tiles_width = 1024    #wedth of the image
tiles_height = 1024   #heigh of the image
output_filename = 'tile_{}-{}.tif'
#input_filename='1.tif'
#inputArg = sys.argv

#out_path="/home/nteupe/extractBuildings/preprocessing/output/"
#in_path="/home/nteupe/extractBuildings/preprocessing/"








#inputArg = sys.argv
#ou_path="output/clip/"
#os.mkdir(ou_path+inputArg[1])
#out_path=ou_path+inputArg[1] + '/'
#in_path="input/"= inputArg[1]+'/'
#input_filename=inputArg[1]
#tiles_width = 256
#tiles_height = 256

def get_tiles(ds, width = tiles_width, height = tiles_height):
    nols, nrows = ds.meta['width'], ds.meta['height']
    offsets = product(range(0, nols, width), range(0, nrows, height))
    big_window = windows.Window(col_off=0, row_off=0, width=nols, height=nrows)
    for col_off, row_off in  offsets:
        window =windows.Window(col_off=col_off, row_off=row_off, width=width, height=height).intersection(big_window)
        transform = windows.transform(window, ds.transform)
        yield window, transform

class Tiles:
    def cut(in_path, input_filename, out_path,):
        """ Slice the images into several images of equal dimensions.
    
        Arguments
        ---------
        in_path : Str
            Path to the image.
        input_filename:str
            Name of the image
        save_path : Str
            Path to sav the masked slice images .

        Returns
        -------
        Several image of different dimension.

        Notes
        -----
        This functions depends on "rasterio" with it's subfunctions.
        """
        with rio.open(os.path.join(in_path, input_filename)) as inds:
            output_filename = 'tile_{}-{}.tif'
            tiles_width = 1024    #wedth of the image
            tiles_height = 1024
            tile_width, tile_height = tiles_width, tiles_height

            meta = inds.meta.copy()

            for window, transform in get_tiles(inds):
                print(window)
                meta['transform'] = transform
                meta['width'], meta['height'] = window.width, window.height
                outpath = os.path.join(out_path,output_filename.format(int(window.col_off), int(window.row_off)))
                with rio.open(outpath, 'w', **meta) as outds:
                    outds.write(inds.read(window=window))
        print('ok') 


In [ ]:
from osgeo import gdal
import sys
import os
from os import listdir
from os.path import join
from os.path import isfile
from tkinter import *
from tkinter import Tcl





class Transfer_info:
    def Giveninfor(image_path, mask_path):
        "This function is to transfer geometric informationfrom one raster to another"
        output= [f for f in listdir(mask_path) if isfile(join(mask_path, f)) and  f.endswith(".tif")]
        input= [f for f in listdir(image_path) if isfile(join(image_path, f)) and  f.endswith(".tif")]
        inp=Tcl().call('lsort', '-dict',input)  
        out=Tcl().call('lsort', '-dict',output)
        print(len(inp))
        print(len(out))
        a=[]        #list the slide images
        b=[]       # list for the mask
        for file in inp:
            a.append(gdal.Open(image_path + file, gdal.GA_ReadOnly))            
        for file in out:
            b.append(gdal.Open(mask_path + file, gdal.GA_Update))
        for i in range(len(a)):
            a[i].GetProjection()
            a[i].GetGeoTransform()
            b[i].SetProjection(a[i].GetProjection())          #setting the mask  projection
            b[i].SetGeoTransform(a[i].GetGeoTransform())      #setting the mask trnsformatio
            b[i]= None                          #saving the image link


In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import json
import os
import shutil
import zipfile

# Root directory of the project
ROOT_DIR = os.path.abspath('C:\\Users\\Steven\\Downloads\\new_automation-20220128T083924Z-001\\new_automation\\Mask_RCNN')
print("VERS 0.2")
# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from PIL import Image, ImageDraw

import warnings
warnings.filterwarnings('ignore')

In [ ]:

# General Workflow

In [ ]:
import cv2
#from patchify import patchify, unpatchify
import tensorflow as tf


In [ ]:
class_number = 1

config = CustomConfig(class_number)
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)
model.load_weights("C:/Users/Steven/Downloads/new_automation-20220128T083924Z-001/new_automation/saved_model/mask_rcnn_object_0015.h5", by_name=True)

In [ ]:
#Apply a trained model on large image
import matplotlib.pyplot as plt
test_image = cv2.imread('/content/drive/My Drive/plot_delineation/tile_8.tif') #BGR
test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)
print(test_image.shape)
#Crop the image to be divisible by 1024
patches_size = 1024
test_image = cv2.resize(test_image, (test_image.shape[1]//1024 * patches_size,test_image.shape[0]//1024 * patches_size))
print(test_image.shape)
#This will split the image into small images of shape [3,3]
#patches = patchify(test_image, (1024,1024, 3), step=1024)  #Step=1024 for 1024 patches means no overlap
plt.imshow(test_image)

In [ ]:
image1 = mpimg.imread('/content/drive/My Drive/plot_delineation/Copy of tile_0_3072.jpg')

def predict_image(tile_image):

  results = model.detect([tile_image])
  mask_generated = results[0]['masks']
  masked_img=np.any(mask_generated.astype(np.bool), axis=-1)
  #convert it to 4D
  #masked_img = np.expand_dims(masked_img, axis=0)
 #masked_img = np.stack((masked_img,)*3, axis=-1)
  return masked_img

predict_image(image1).shape


In [ ]:
image1.shape

In [ ]:
#res1 = predict_image(image1)

#res1.shape
#res2 = np.expand_dims(res1, axis=0)
#print(res2.shape)
#np.stack((res1,)*3, axis=-1).shape

In [ ]:
import numpy as np
import scipy.signal
from tqdm import tqdm

import gc


if __name__ == '__main__':
    import matplotlib.pyplot as plt
    PLOT_PROGRESS = True
    # See end of file for the rest of the __main__.
else:
    PLOT_PROGRESS = False


def _spline_window(window_size, power=2):
    """
    Squared spline (power=2) window function:
    https://www.wolframalpha.com/input/?i=y%3Dx**2,+y%3D-(x-2)**2+%2B2,+y%3D(x-4)**2,+from+y+%3D+0+to+2
    """
    intersection = int(window_size/4)
    wind_outer = (abs(2*(scipy.signal.triang(window_size))) ** power)/2
    wind_outer[intersection:-intersection] = 0

    wind_inner = 1 - (abs(2*(scipy.signal.triang(window_size) - 1)) ** power)/2
    wind_inner[:intersection] = 0
    wind_inner[-intersection:] = 0

    wind = wind_inner + wind_outer
    wind = wind / np.average(wind)
    return wind


cached_2d_windows = dict()
def _window_2D(window_size, power=2):
    """
    Make a 1D window function, then infer and return a 2D window function.
    Done with an augmentation, and self multiplication with its transpose.
    Could be generalized to more dimensions.
    """
    # Memoization
    global cached_2d_windows
    key = "{}_{}".format(window_size, power)
    if key in cached_2d_windows:
        wind = cached_2d_windows[key]
    else:
        wind = _spline_window(window_size, power)
        wind = np.expand_dims(np.expand_dims(wind, 1), 1)      #SREENI: Changed from 3, 3, to 1, 1 
        wind = wind * wind.transpose(1, 0, 2)
        if PLOT_PROGRESS:
            # For demo purpose, let's look once at the window:
            plt.imshow(wind[:, :, 0], cmap="viridis")
            plt.title("2D Windowing Function for a Smooth Blending of "
                      "Overlapping Patches")
            plt.show()
        cached_2d_windows[key] = wind
    return wind


def _pad_img(img, window_size, subdivisions):
    """
    Add borders to img for a "valid" border pattern according to "window_size" and
    "subdivisions".
    Image is an np array of shape (x, y, nb_channels).
    """
    aug = int(round(window_size * (1 - 1.0/subdivisions)))
    more_borders = ((aug, aug), (aug, aug), (0, 0))
    ret = np.pad(img, pad_width=more_borders, mode='reflect')
    # gc.collect()

    if PLOT_PROGRESS:
        # For demo purpose, let's look once at the window:
        plt.imshow(ret)
        plt.title("Padded Image for Using Tiled Prediction Patches\n"
                  "(notice the reflection effect on the padded borders)")
        plt.show()
    return ret


def _unpad_img(padded_img, window_size, subdivisions):
    """
    Undo what's done in the `_pad_img` function.
    Image is an np array of shape (x, y, nb_channels).
    """
    aug = int(round(window_size * (1 - 1.0/subdivisions)))
    ret = padded_img[
        aug:-aug,
        aug:-aug,
        :
    ]
    # gc.collect()
    return ret


def _rotate_mirror_do(im):
    """
    Duplicate an np array (image) of shape (x, y, nb_channels) 8 times, in order
    to have all the possible rotations and mirrors of that image that fits the
    possible 90 degrees rotations.
    It is the D_4 (D4) Dihedral group:
    https://en.wikipedia.org/wiki/Dihedral_group
    """
    mirrs = []
    mirrs.append(np.array(im))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=1))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=2))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=3))
    im = np.array(im)[:, ::-1]
    mirrs.append(np.array(im))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=1))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=2))
    mirrs.append(np.rot90(np.array(im), axes=(0, 1), k=3))
    return mirrs


def _rotate_mirror_undo(im_mirrs):
    """
    merges a list of 8 np arrays (images) of shape (x, y, nb_channels) generated
    from the `_rotate_mirror_do` function. Each images might have changed and
    merging them implies to rotated them back in order and average things out.
    It is the D_4 (D4) Dihedral group:
    https://en.wikipedia.org/wiki/Dihedral_group
    """
    origs = []
    origs.append(np.array(im_mirrs[0]))
    origs.append(np.rot90(np.array(im_mirrs[1]), axes=(0, 1), k=3))
    origs.append(np.rot90(np.array(im_mirrs[2]), axes=(0, 1), k=2))
    origs.append(np.rot90(np.array(im_mirrs[3]), axes=(0, 1), k=1))
    origs.append(np.array(im_mirrs[4])[:, ::-1])
    origs.append(np.rot90(np.array(im_mirrs[5]), axes=(0, 1), k=3)[:, ::-1])
    origs.append(np.rot90(np.array(im_mirrs[6]), axes=(0, 1), k=2)[:, ::-1])
    origs.append(np.rot90(np.array(im_mirrs[7]), axes=(0, 1), k=1)[:, ::-1])
    return np.mean(origs, axis=0)


def _windowed_subdivs(padded_img, window_size, subdivisions, nb_classes, pred_func):
    """
    Create tiled overlapping patches.
    Returns:
        5D numpy array of shape = (
            nb_patches_along_X,
            nb_patches_along_Y,
            patches_resolution_along_X,
            patches_resolution_along_Y,
            nb_output_channels
        )
    Note:
        patches_resolution_along_X == patches_resolution_along_Y == window_size
    """
    WINDOW_SPLINE_2D = _window_2D(window_size=window_size, power=2)
    
    step = int(window_size/subdivisions)
    padx_len = padded_img.shape[0]
    pady_len = padded_img.shape[1]
    subdivs = []
    
    for i in range(0, padx_len-window_size+1, step):
        subdivs.append([])
        for j in range(0, pady_len-window_size+1, step):            #SREENI: Changed padx to pady (Bug in original code)
            patch = padded_img[i:i+window_size, j:j+window_size, :]
            subdivs[-1].append(patch)
   
    # Here, `gc.collect()` clears RAM between operations.
    # It should run faster if they are removed, if enough memory is available.
    gc.collect()
    subdivs = np.array(subdivs)
    gc.collect()
    a, b, c, d, e = subdivs.shape
    subdivs = subdivs.reshape(a * b, c, d, e)
    gc.collect()

    subdivs = pred_func(subdivs)
    gc.collect()

    print(subdivs)
    print(subdivs.shape)
    subdivs = np.array([patch * WINDOW_SPLINE_2D for patch in subdivs])
    gc.collect()

    # Such 5D array:
    subdivs = subdivs.reshape(a, b, c, d, nb_classes)
    gc.collect()

    return subdivs


def _recreate_from_subdivs(subdivs, window_size, subdivisions, padded_out_shape):
    """
    Merge tiled overlapping patches smoothly.
    """
    step = int(window_size/subdivisions)
    padx_len = padded_out_shape[0]
    pady_len = padded_out_shape[1]

    y = np.zeros(padded_out_shape)

    a = 0
    for i in range(0, padx_len-window_size+1, step):
        b = 0
        for j in range(0, pady_len-window_size+1, step):                #SREENI: Changed padx to pady (Bug in original code)
            windowed_patch = subdivs[a, b]
            y[i:i+window_size, j:j+window_size] = y[i:i+window_size, j:j+window_size] + windowed_patch
            b += 1
        a += 1
    return y / (subdivisions ** 2)


def predict_img_with_smooth_windowing(input_img, window_size, subdivisions, nb_classes, pred_func):
    """
    Apply the `pred_func` function to square patches of the image, and overlap
    the predictions to merge them smoothly.
    See 6th, 7th and 8th idea here:
    http://blog.kaggle.com/2017/05/09/dstl-satellite-imagery-competition-3rd-place-winners-interview-vladimir-sergey/
    """
    pad = _pad_img(input_img, window_size, subdivisions)
  

    pads = _rotate_mirror_do(pad)
   

    # Note that the implementation could be more memory-efficient by merging
    # the behavior of `_windowed_subdivs` and `_recreate_from_subdivs` into
    # one loop doing in-place assignments to the new image matrix, rather than
    # using a temporary 5D array.

    # It would also be possible to allow different (and impure) window functions
    # that might not tile well. Adding their weighting to another matrix could
    # be done to later normalize the predictions correctly by dividing the whole
    # reconstructed thing by this matrix of weightings - to normalize things
    # back from an impure windowing function that would have badly weighted
    # windows.

    # For example, since the U-net of Kaggle's DSTL satellite imagery feature
    # prediction challenge's 3rd place winners use a different window size for
    # the input and output of the neural net's patches predictions, it would be
    # possible to fake a full-size window which would in fact just have a narrow
    # non-zero dommain. This may require to augment the `subdivisions` argument
    # to 4 rather than 2.
    res = []
    for pad in tqdm(pads):
        # For every rotation:
        sd = _windowed_subdivs(pad, window_size, subdivisions, nb_classes, pred_func)
        one_padded_result = _recreate_from_subdivs(
            sd, window_size, subdivisions,
            padded_out_shape=list(pad.shape[:-1])+[nb_classes])
        
        res.append(one_padded_result)

    # Merge after rotations:
    padded_results = _rotate_mirror_undo(res)

    prd = _unpad_img(padded_results, window_size, subdivisions)

    prd = prd[:input_img.shape[0], :input_img.shape[1], :]

    if PLOT_PROGRESS:
        plt.imshow(prd)
        plt.title("Smoothly Merged Patches that were Tiled Tighter")
        plt.show()
     
    return prd


In [ ]:
import cv2
import numpy as np

from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


                  
# size of patches
patch_size = 1024

# Number of classes 
n_classes = 3

         
###################################################################################
#Predict using smooth blending

# Use the algorithm. The `pred_func` is passed and will process all the image 8-fold by tiling small patches with overlap, called once with all those image as a batch outer dimension.
# Note that model.predict(...) accepts a 4D tensor of shape (batch, x, y, nb_channels), such as a Keras model.
predictions_smooth = predict_img_with_smooth_windowing(
    test_image,
    window_size=patch_size,
    subdivisions=2,  # Minimal amount of overlap for windowing. Must be an even number.
    nb_classes=n_classes,
    pred_func=(
        lambda img_batch_subdiv: predict_image((img_batch_subdiv))
    )
)

In [ ]:

pad = _pad_img(test_image,  patch_size,  2)

pads = _rotate_mirror_do(pad)

pads[0].shape

In [ ]:

res = []
for pad in tqdm(pads):
    # For every rotation:
    sd = _windowed_subdivs(pad, window_size, subdivisions, nb_classes, pred_func)
    one_padded_result = _recreate_from_subdivs(
        sd, window_size, subdivisions,
        padded_out_shape=list(pad.shape[:-1])+[nb_classes])
    
    res.append(one_padded_result)

# Merge after rotations:
padded_results = _rotate_mirror_undo(res)

prd = _unpad_img(padded_results, window_size, subdivisions)

prd = prd[:input_img.shape[0], :input_img.shape[1], :]

if PLOT_PROGRESS:
    plt.imshow(prd)
    plt.title("Smoothly Merged Patches that were Tiled Tighter")
    plt.show()


In [ ]:
test_image.shape

In [ ]:
#_windowed_subdivs(pads[0], patch_size, 2, 1, model)

padded_img = pads[0]
window_size = patch_size
subdivisions = 2
nb_classes = 3
WINDOW_SPLINE_2D = _window_2D(window_size, power=2)
print(WINDOW_SPLINE_2D.shape)
step = int(window_size/subdivisions)
padx_len = padded_img.shape[0]
pady_len = padded_img.shape[1]
print((step,padx_len,pady_len))
subdivs = []
for i in range(0, padx_len-window_size+1, step):
        subdivs.append([])
        for j in range(0, pady_len-window_size+1, step):            #SREENI: Changed padx to pady (Bug in original code)
            patch = padded_img[i:i+window_size, j:j+window_size, :]
            subdivs[-1].append(patch)

#gc.collect()
subdivs = np.array(subdivs)
#gc.collect()
a, b, c, d, e = subdivs.shape
subdivs = subdivs.reshape(a * b, c, d, e)
#gc.collect()
print(subdivs[0].shape)



In [ ]:

subdivs = map(lambda subdivs : predict_image(subdivs), subdivs)
subdivs
gc.collect()




In [ ]:

subdivs = np.array([patch * WINDOW_SPLINE_2D for patch in subdivs])
gc.collect()


In [ ]:
# Such 5D array:
subdivs = subdivs.reshape(a, b, c, d, nb_classes)
gc.collect()

In [ ]:
subdivs1.reshape(9, 11, 1024, 1024, 3)

In [ ]:
#subdivs1 = subdivs[0:2,:,:]
#for patch in subdivs1:
 # patch = patch

#patch.shape, WINDOW_SPLINE_2D.shape
#patch * WINDOW_SPLINE_2D
subdivs1 = subdivs1.reshape(2, 1, c, d, 3)

In [ ]:
a, b, c, d
subdivs1.shape,subdivs.shape,a, b, c, d

In [ ]:

arr = np.array([[2,16,3,4],[1,30, 4, 16, 64],[17,3, 6, 9, 12]])
#(map(lambda arr : sorted(arr), arr))

# Sort each sublist
#list(lambda arr: (sorted(i) for i in arr[i]))
#[predict_image(arr) for i,arr in enumerate(subdivs)]

#dd = list(map(lambda subdivs : predict_image(subdivs), subdivs))
np.array(dd)

In [ ]:
List = [[2,3,4],[1, 4, 16, 64],[3, 6, 9, 12]]
 
# Sort each sublist
sortList = lambda x: (sorted(i) for i in x)
print((sortList))

In [ ]:
list(lambda subdivs: (predict_image(i) for i in subdivs[i]))

In [ ]:


subdivs = pred_func(subdivs)
gc.collect()


subdivs = np.array([patch * WINDOW_SPLINE_2D for patch in subdivs])
gc.collect()

# Such 5D array:
subdivs = subdivs.reshape(a, b, c, d, nb_classes)
gc.collect()


In [ ]:
test_image.shape

In [ ]:
import numpy as np
final_prediction = np.argmax(predictions_smooth, axis=2)

In [ ]:

plt.figure(figsize=(12, 12))
plt.subplot(221)
plt.title('Testing Image')
plt.imshow(test_image)
plt.subplot(222)
plt.title('Testing Label')
plt.imshow(test_image)
plt.subplot(223)
plt.title('Prediction with smooth blending')
plt.imshow(final_prediction)
plt.show()

In [ ]:
io.imsave('/content/drive/My Drive/plot_delineation/prediction_test1.jpg',reconstructed_image)